In [2]:
import pandas as pd

In [12]:
input_data = "../5a-Matching/output/matched.csv"
df = pd.read_csv(input_data, low_memory=False, dtype="str")

In [13]:
len(df)

803490

In [22]:
len(df[df.latitude.isna() & df.x.isna()])

333236

In [25]:
sample_size = 50
if (len(df) > sample_size):
    df = df.sample(sample_size)
else:
    sample_size = len(df)

In [28]:
# we need to geocode if a results has na for latitude AND has ratio < 80 OR ratio na

# ok how to get started with geocoding. 
# let's just test the bing script

